In [ ]:
from collections import Counter
import os
import numpy as np
import torch
import torch_geometric
import spark_dsg as dsg

## Load dataset file

In [ ]:
# dataset file path
hydra_dataset_dir = "/mnt/sdb/dataset/tmp/tro_graphs_2022_09_24"
trajectory_dirs = os.listdir(hydra_dataset_dir)

In [ ]:
scene_counter = Counter(full_name.split("_")[0] for full_name in trajectory_dirs)
scene_names = list(scene_counter)
print("Found {} scenes.".format(len(scene_names)))
print("Number of samples for each scene:\n", dict(scene_counter))

In [ ]:
# Pick a random scene graph file
test_scene = scene_names[0]
test_num_frames = 600

test_json_file = f"{hydra_dataset_dir}/{test_scene}_trajectory_0/gt_partial_dsg_{test_num_frames}.json"
assert os.path.exists(test_json_file)
print(test_json_file)

In [ ]:
# Load scene graph
G = dsg.DynamicSceneGraph.load(test_json_file)
assert G.num_nodes() > 0
assert G.num_edges() > 0

# access nodes in each layer
print("Number of nodes separated by layer: {} ({} total).".format([layer.num_nodes() for layer in G.layers], G.num_nodes()))

In [ ]:
# Access node attributes at specified layer
# (room nodes example)
print("Raw room attributes.")
print([node.attributes for node in G.get_layer(dsg.DsgLayers.ROOMS).nodes])

In [ ]:
# Visualize scene graph with open3d (if installed)
dsg.render_to_open3d(G)

## Helpful function - add room bounding box from children places nodes

In [ ]:
# This room bounding box is the the smallest axis-aligned box containing all children places nodes
dsg.add_bounding_boxes_to_layer(G, dsg.DsgLayers.ROOMS)
print("Updated room attributes (check bounding box).")
print([node.attributes for node in G.get_layer(dsg.DsgLayers.ROOMS).nodes])

## Helpful function - replace hydra room segmentation with ground truth mp3d segmentation

In [ ]:
from hydra_gnn.mp3d import load_mp3d_info, repartition_rooms

# mp3d dataset directory
mp3d_dataset_dir = "/mnt/sdb/dataset/Matterport3D/v1/scans"
gt_house_file = f"{mp3d_dataset_dir}/{test_scene}/house_segmentations/{test_scene}.house"
assert os.path.exists(gt_house_file)
print(gt_house_file)

In [ ]:
# Load ground truth house segmentation file, generate a DynamicSceneGraph with ground truth room segementation,
# and connect places node to the new room nodes.
gt_house_info = load_mp3d_info(gt_house_file)
G_update = repartition_rooms(G, gt_house_info)

## Helpful function - convert DynamicSceneGraph to pytorch geometric graph

In [ ]:
from spark_dsg.torch_conversion import _centroid_bbx_embedding

# Homogeneous graph
data_homogeneous = G.to_torch(_centroid_bbx_embedding, use_heterogeneous=False)
print("Homogenous torch graph:")
print(data_homogeneous)

# Heterogeneous graph
data_heterogeneous = G.to_torch(_centroid_bbx_embedding, use_heterogeneous=True)
print("\nHeterogeneous torch graph:")
print(data_heterogeneous)
print('node types:', data_heterogeneous.node_types)
print('edge types:', data_heterogeneous.edge_types)